# Тема 1. Интерполирование функции с помощью многочленов Лагранжа и многочленов Ньютона с разделенными разностями

In [11]:
import numpy as np
import math
import pandas as pd

Работать с математическим наименованием в питоне было очень сложно, поэтому мне пришлось его немного адаптировать \
x_ip1 -> x_{i+1} \
x_im1 -> x_{i-1} 

X_star_1 -> x^{*} \
X_star_2 -> x^{**} 

f_d1(x) -> f'(x) \
f_d2(x) -> f''(x) 

### мой вариант 4
$y=(x-1)^2-0.5e^x$ \
$[a,b] = [0.1, 0.6]$ \
$x^{*} = 0.13$ \
$x^{**} = 0.12$ \
$x^{***} = 0.57$ \
$x^{****} = 0.33$

In [12]:
def f(x):
    return (x-1)**2 - 0.5 * math.exp(x)

A = 0.1
B = 0.6
X_star_1 = 0.13
X_star_2 = 0.12
X_star_3 = 0.57
X_star_4 = 0.33 

X_star = X_star_1

In [13]:
def f_d2(x):
    return 2 - 0.5 * math.exp(x)

def f_d3(x):
    return -0.5 * math.exp(x)

1. Построить таблицу из 11 значений выбранной функции.

In [14]:
table_parameters = np.linspace(A, B, 11)
data_table = {"x_i": list(table_parameters), "f(x_i)": [f(i) for i in table_parameters]}
pd.DataFrame(data=data_table)

,x_i,f(x_i)
0,0.10,0.257415
1,0.15,0.141583
2,0.20,0.029299
3,0.25,-0.079513
4,0.30,-0.184929
5,0.35,-0.287034
6,0.40,-0.385912
7,0.45,-0.481656
8,0.50,-0.574361
9,0.55,-0.664127


2. По интерполяционной формуле Лагранжа первого порядка вычислить

In [15]:
i, x_i, x_ip1 = 0, 0, 0

temp_i = 0
for x_1, x_2 in zip(table_parameters[:-2], table_parameters[1:]):
    if x_1 <= X_star <= x_2:
        x_i, x_ip1 = x_1, x_2
        i = temp_i
        break
    temp_i += 1

def L_1(x, x_i, x_ip1):
    return f(x_i)*(x-x_ip1)/(x_i-x_ip1) + f(x_ip1)*(x-x_i)/(x_ip1-x_i)


L_1_star = L_1(X_star, x_i, x_ip1)
f_star = f(X_star)
R_1_star = L_1_star - f_star

print(f"Лагранж 1-го порядка для x* = {X_star}:")
print(f"Узлы: x_{i} = {x_i:.3f}, x_{i+1} = {x_ip1:.3f}")
print(f"L_1(x*) = {L_1_star:.6f}")
print(f"f(x*) = {f_star:.6f}")
print(f"R_1(x*) = L_1 - f = {R_1_star:.6f}")


Лагранж 1-го порядка для x* = 0.13:
Узлы: x_0 = 0.100, x_1 = 0.150
L_1(x*) = 0.187916
f(x*) = 0.187486
R_1(x*) = L_1 - f = 0.000430


3. Оценка остаточного члена R_1(x)

In [16]:
def omega_2(x, x_i, x_ip1):
    return (x - x_i) * (x - x_ip1)

# функция монотонная
f_d2_min = min(f_d2(x_i), f_d2(x_ip1))
f_d2_max = max(f_d2(x_i), f_d2(x_ip1))

# ω_2 - парабола, максимум/минимум в середине отрезка или на концах
x_mid = (x_i + x_ip1) / 2
omega_2_vals = [omega_2(x_i, x_i, x_ip1), omega_2(x_ip1, x_i, x_ip1), 
               omega_2(x_mid, x_i, x_ip1)]
omega_2_min = min(omega_2_vals)
omega_2_max = max(omega_2_vals)

R_1_min = (f_d2_min * omega_2_min) / 2
R_1_max = (f_d2_max * omega_2_max) / 2

print(f"Оценка R_1(x) на [{x_i:.3f}, {x_ip1:.3f}]:")
print(f"f''_min = {f_d2_min:.6f}, f''_max = {f_d2_max:.6f}")
print(f"ω_2_min = {omega_2_min:.6f}, ω_2_max = {omega_2_max:.6f}")
print(f"min R_1 = {R_1_min:.6f}, max R_1 = {R_1_max:.6f}")
print(f"R_1(x*) = {R_1_star:.6f}")
print(f"min R_1 < R_1(x*) < max R_1: {R_1_min < R_1_star < R_1_max}")


Оценка R_1(x) на [0.100, 0.150]:
f''_min = 1.419083, f''_max = 1.447415
ω_2_min = -0.000625, ω_2_max = -0.000000
min R_1 = -0.000443, max R_1 = -0.000000
R_1(x*) = 0.000430
min R_1 < R_1(x*) < max R_1: False


4. Интерполяция Лагранжа 2-го порядка

In [17]:
# Берём три узла: x_{i-1}, x_i, x_{i+1}

left_i = 0
for i in range(len(table_parameters)):
    if table_parameters[i] >= X_star:
        left_i = i - 1
        break
center_i = left_i + 1
right_i = center_i + 1
if right_i >= len(table_parameters):
    left_i -= 1
    center_i -= 1
    right_i -= 1

x_im1 = table_parameters[left_i]
x_i = table_parameters[center_i]
x_ip1 = table_parameters[right_i]


def L_2(x, x_im1, x_i, x_ip1):
    term1 = f(x_im1) * ((x - x_i) * (x - x_ip1)) / ((x_im1 - x_i) * (x_im1 - x_ip1))
    term2 = f(x_i) * ((x - x_im1) * (x - x_ip1)) / ((x_i - x_im1) * (x_i - x_ip1))
    term3 = f(x_ip1) * ((x - x_im1) * (x - x_i)) / ((x_ip1 - x_im1) * (x_ip1 - x_i))
    return term1 + term2 + term3

L_2_star = L_2(X_star, x_im1, x_i, x_ip1)
R_2_star = L_2_star - f_star

print(f"Лагранж 2-го порядка для x* = {X_star}:")
print(f"Узлы: x_{left_i} = {x_im1:.3f}, x_{center_i} = {x_i:.3f}, x_{right_i} = {x_ip1:.3f}")
print(f"L_2(x*) = {L_2_star:.6f}")
print(f"R_2(x*) = L_2 - f = {R_2_star:.6f}")

Лагранж 2-го порядка для x* = 0.13:
Узлы: x_0 = 0.100, x_1 = 0.150, x_2 = 0.200
L_2(x*) = 0.187490
R_2(x*) = L_2 - f = 0.000004


5. Оценка остаточного члена R₂(x)

In [18]:
def omega_3(x, x_im1, x_i, x_ip1):
    return (x - x_im1) * (x - x_i) * (x - x_ip1)

f_d3_min = min(f_d3(x_im1), f_d3(x_ip1))
f_d3_max = max(f_d3(x_im1), f_d3(x_ip1))

# критические точки кубического уравнения
x_crit1 = (2*x_im1 + x_i + x_ip1) / 4
x_crit2 = (x_im1 + 2*x_i + x_ip1) / 4
x_crit3 = (x_im1 + x_i + 2*x_ip1) / 4

omega_3_vals = [omega_3(x_im1, x_im1, x_i, x_ip1),
               omega_3(x_i, x_im1, x_i, x_ip1),
               omega_3(x_ip1, x_im1, x_i, x_ip1),
               omega_3(x_crit1, x_im1, x_i, x_ip1),
               omega_3(x_crit2, x_im1, x_i, x_ip1),
               omega_3(x_crit3, x_im1, x_i, x_ip1)]

omega_3_min = min(omega_3_vals)
omega_3_max = max(omega_3_vals)

R_2_min = (f_d3_min * omega_3_min) / 6
R_2_max = (f_d3_max * omega_3_max) / 6

print(f"Оценка R_2(x) на [{x_im1:.3f}, {x_ip1:.3f}]:")
print(f"f'''_min = {f_d3_min:.6f}, f'''_max = {f_d3_max:.6f}")
print(f"ω_3_min = {omega_3_min:.6f}, ω_3_max = {omega_3_max:.6f}")
print(f"min R_2 = {R_2_min:.6f}, max R_2 = {R_2_max:.6f}")
print(f"R_2(x*) = {R_2_star:.6f}")
print(f"min R_2 < R_2(x*) < max R_2: {R_2_min < R_2_star < R_2_max}")

Оценка R_2(x) на [0.100, 0.200]:
f'''_min = -0.610701, f'''_max = -0.552585
ω_3_min = -0.000029, ω_3_max = 0.000029
min R_2 = 0.000003, max R_2 = -0.000003
R_2(x*) = 0.000004
min R_2 < R_2(x*) < max R_2: False


6. Интерполяция Ньютона

In [19]:
# функции нахождения разделенных разностей
def divided_diff(*params):
    if len(params) == 2:
        x_i, x_ip1 = params
        return (f(x_ip1) - f(x_i)) / (x_ip1 - x_i)
    if len(params) == 3:
        x_im1, x_i, x_ip1 = params
        return (divided_diff(x_i, x_ip1) - divided_diff(x_im1, x_i)) / (x_ip1 - x_im1)

def N_1(x, x_i, x_ip1):
    return f(x_i) + divided_diff(x_i, x_ip1) * (x - x_i)

def N_2(x, x_im1, x_i, x_ip1):
    return f(x_im1) + divided_diff(x_im1, x_i) * (x - x_im1) + divided_diff(x_im1, x_i, x_ip1) * (x - x_im1) * (x - x_i)

N_1_star = N_1(X_star, x_i, x_ip1)
N_2_star = N_2(X_star, x_im1, x_i, x_ip1)

print("Разделенные разности:")
print(f"f[x_{i-1}, x_{i}] = {divided_diff(x_im1, x_i):.6f}")
print(f"f[x_{i}, x_{i+1}] = {divided_diff(x_i, x_ip1):.6f}")
print(f"f[x_{i-1}, x_{i}, x_{i+1}] = {divided_diff(x_im1, x_i, x_ip1):.6f}")
print(f"N_1(x*) = {N_1_star:.6f} (погрешность: {abs(N_1_star - L_1_star):.3e})")
print(f"N_2(x*) = {N_2_star:.6f} (погрешность: {abs(N_2_star - L_2_star):.3e})")

Разделенные разности:
f[x_0, x_1] = -2.316633
f[x_1, x_2] = -2.245685
f[x_0, x_1, x_2] = 0.709481
N_1(x*) = 0.186497 (погрешность: 1.419e-03)
N_2(x*) = 0.187490 (погрешность: 0.000e+00)


погрешность по сравнению с действительным результатом

In [20]:
real = f(X_star)

print(f"L_1(x*) (погрешность: {abs(L_1_star - real):.3e})")
print(f"L_2(x*) (погрешность: {abs(L_2_star - real):.3e})")
print(f"N_1(x*) (погрешность: {abs(N_1_star - real):.3e})")
print(f"N_2(x*) (погрешность: {abs(N_2_star - real):.3e})")

L_1(x*) (погрешность: 4.297e-04)
L_2(x*) (погрешность: 4.047e-06)
N_1(x*) (погрешность: 9.892e-04)
N_2(x*) (погрешность: 4.047e-06)


ссылка на эту работу на Github
https://github.com/matv864/AI_work/tree/main/computational_math/LR2.ipynb